### Generating game character with generative adversarial networks

Training a generative adversarial network on the  0x72.itch.io-scraped dataset of 185472 16 × 16 pixel 2-bit gameboy-character-like images to generate characters.


In [1]:
import random
from typing import Tuple
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision.transforms import ToPILImage
from torch.utils.data import Dataset, DataLoader

<class 'ModuleNotFoundError'>: No module named 'torch'

In [10]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
gbc_ndarray = np.load("./gbc_dataset.npy")

class GBCDataset(Dataset):
    """
    Implement gameboy character dataset (GBCDataset) class.

    Attributes:
        dat (np.ndarray): Data.
        dev (str): Device.
    """
    def __init__(self, data, device):
        """
        Instantiate GBCDataset class.

        Args:
            data (np.ndarray): Data.
            device (str): Device.
        """
        self.dat = data
        self.dev = device

    def __len__(self):
        """
        Get dataset cardinality.

        Returns:
            int: Dataset cardinality.
        """
        return self.dat.shape[0]

    def __getitem__(self, index):
        """
        Get dataset element.

        Args:
            index (int): Index.

        Returns:
            torch.Tensor: Dataset element.
        """
        return torch.from_numpy(self.dat[index].astype(np.float32) / 127.5 - 1.0).to(self.dev)

gbc_dataset = GBCDataset(gbc_ndarray,device)
gbc_data_loader = DataLoader(gbc_dataset, 42, True)

--2023-05-24 17:10:05--  https://umuguc.github.io/file-sharing/gbc_dataset.npy.zip

Resolving umuguc.github.io (umuguc.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...

Connecting to umuguc.github.io (umuguc.github.io)|185.199.108.153|:443... connected.

HTTP request sent, awaiting response... 200 OK

Length: 925054 (903K) [application/zip]

Saving to: ‘gbc_dataset.npy.zip’



gbc_dataset.npy.zip 100%[===================>] 903.37K  --.-KB/s    in 0.006s  



2023-05-24 17:10:05 (138 MB/s) - ‘gbc_dataset.npy.zip’ saved [925054/925054]



Archive:  gbc_dataset.npy.zip

  inflating: assignment_5/gbc_dataset.npy  

  inflating: assignment_5/__MACOSX/._gbc_dataset.npy  


In [102]:
class GBCGAN:
    """
    Implement gameboy character generative adversarial network (GBCGAN) class.

    Attributes:
        _dis_net (nn.Sequential): Discriminator network.
        _dis_opt (optim.Optimizer): Discriminator optimiser.
        _gen_net (nn.Sequential): Generator network.
        _gen_opt (optim.Optimizer): Generator optimiser.
        _lat_dim (int): Latent dimensionality.
        _los_fun (nn.Loss): Loss function.
        dev (str): Device.

    Methods:
        _get_discriminator(): Get discriminator network and discriminator optimiser.
        _get_generator(): Get generator network and generator optimiser.
        generate_images(batch_size): Generate fake images.
        train_networks(dataset, epoch_num, batch_size): Train discriminator network and generator network.
    """

    def __init__(self, device: str = "cuda") -> None:
        """
        Instantiate GBCGAN class.

        Args:
            device (str): Device.
        """
        self.dev = device
        self._dis_net, self._dis_opt = self._get_discriminator()
        self._gen_net, self._gen_opt = self._get_generator()
        self._lat_dim = self._gen_net[0].in_channels
        self._los_fun = nn.BCELoss()

    def _get_discriminator(self) -> Tuple[nn.Sequential, optim.Optimizer]:
        """
        Get discriminator network and discriminator optimiser.

        Returns:
            network (nn.Sequential): Discriminator network.
            optimiser (optim.Optimizer): Discriminator optimiser.
        """
        network = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=4, stride=2, padding=1,bias=False),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Conv2d(16,32,kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Conv2d(32,1,kernel_size=4, stride=1, padding=1, bias=False),
            nn.Sigmoid()
        ).to(self.dev)
        
        optimiser = optim.Adam(network.parameters(), lr=0.0002, betas=(0.5, 0.999))

        return network, optimiser

    def _get_generator(self) -> Tuple[nn.Sequential, optim.Optimizer]:
        """
        Get generator network and generator optimiser.

        Returns:
            network (nn.Sequential): Generator network.
            optimiser (optim.Optimizer): Generator optimiser.
        """
        
        network = nn.Sequential(
            nn.ConvTranspose2d(64,32,kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32,16,kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.ConvTranspose2d(16,1,kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        ).to(self.dev)

        optimiser = optim.Adam(network.parameters(), lr=0.0002, betas=(0.5, 0.999))

        return network, optimiser

    def generate_images(self, batch_size):
        """
        Generate fake images.

        Args:
            batch_size (int): Batch size.

        Returns:
            fake_images (PIL.Image): Fake images.
        """
        random_latents = torch.randn(batch_size, self._lat_dim,1,1, device=self.dev)
        temp_images = self._gen_net(random_latents)
        to_pil = ToPILImage()

        fake_images = []

        for i in range(batch_size):
            fake_image = to_pil(temp_images[i].cpu().detach())
            fake_images.append(fake_image)

        final_image = Image.new("L", (16 * batch_size, 16))

        for i, image in enumerate(fake_images):
            final_image.paste(image, (16 * i, 0))

        return final_image

    def train_networks(self, gbc_data_loader: DataLoader, epoch_number: int=1) -> None:
        """
        Train discriminator network and generator network.

        Args:
            gbc_data_loader (DataLoader): GBC data loader.
            epoch_number (int): Epochs number.
        """
        self._dis_net.train()
        self._gen_net.train()
        for epoch in range(epoch_number):
            discriminator_losses = []
            generator_losses = []

            for real_images in gbc_data_loader:

                random_latents =torch.randn(real_images.shape[0], self._lat_dim,1,1, device=self.dev)
                fake_images = self._gen_net(random_latents)
                fake_probs =self._dis_net(fake_images)
                fake_labels =torch.zeros(fake_probs.shape)
                
                real_probs =self._dis_net(real_images)
                real_labels =torch.ones(real_probs.shape)

                discriminator_loss = self._los_fun(fake_probs,fake_labels)+self._los_fun(real_probs,real_labels)
                self._dis_opt.zero_grad()
                discriminator_loss.backward()
                self._dis_opt.step()

                discriminator_losses.append(discriminator_loss.item())

                random_latents =torch.randn(real_images.shape[0], self._lat_dim,1,1, device=self.dev)
                fake_images =self._gen_net(random_latents)
                fake_probs =self._dis_net(fake_images)
                real_labels =torch.ones_like(fake_probs)

                generator_loss =self._los_fun(fake_probs, real_labels)
                self._gen_opt.zero_grad()
                generator_loss.backward()
                self._gen_opt.step()

                generator_losses.append(generator_loss.item())

                print(f"Epoch: [{epoch + 1}/{epoch_number}]")
                print(f"Discriminator loss: {np.mean(discriminator_losses):.4f}")
                print(f"Generator loss: {np.mean(generator_losses):.4f}")

        self._dis_net.eval()
        self._gen_net.eval()

gbcgan = GBCGAN(device)

Training the network :

In [61]:
gbcgan.train_networks(gbc_data_loader,10)


Streaming output truncated to the last 5000 lines.

Discriminator loss: 0.0333

Generator loss: 9.6451

Epoch: [10/10]

Discriminator loss: 0.0333

Generator loss: 9.6451

Epoch: [10/10]

Discriminator loss: 0.0333

Generator loss: 9.6453

Epoch: [10/10]

Discriminator loss: 0.0333

Generator loss: 9.6458

Epoch: [10/10]

Discriminator loss: 0.0333

Generator loss: 9.6462

Epoch: [10/10]

Discriminator loss: 0.0333

Generator loss: 9.6460

Epoch: [10/10]

Discriminator loss: 0.0333

Generator loss: 9.6456

Epoch: [10/10]

Discriminator loss: 0.0333

Generator loss: 9.6460

Epoch: [10/10]

Discriminator loss: 0.0333

Generator loss: 9.6460

Epoch: [10/10]

Discriminator loss: 0.0332

Generator loss: 9.6459

Epoch: [10/10]

Discriminator loss: 0.0332

Generator loss: 9.6459

Epoch: [10/10]

Discriminator loss: 0.0332

Generator loss: 9.6459

Epoch: [10/10]

Discriminator loss: 0.0332

Generator loss: 9.6460

Epoch: [10/10]

Discriminator loss: 0.0332

Generator loss: 9.6455

Epoch: [10/1

In [104]:
fake_images = gbcgan.generate_images(42)

fake_images.show()
fake_images.save("./fake_images.png")

TypeError: ignored